In [ ]:
import pytorch-template as project
gpus = project.assign_free_gpus(threshold_vram_usage=1500)
%env CUDA_VISIBLE_DEVICES=$gpus

In [ ]:
# General Imports
%load_ext autoreload
%autoreload 2

from pathlib import Path

In [ ]:
data_path = Path(project.__path__[0]).parent / "data"